In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=481be8e10888e2628e767bb743de2297caafcb158b64b8ab41a16b25977d95cc
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [97]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import hour, dayofweek, month, col, count, when, countDistinct, min, max, round

In [4]:
spark = SparkSession.builder.appName("CSVTransformation").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/31 17:26:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.csv("/kaggle/input/us-accidents/US_Accidents_March23.csv", inferSchema=True, header=True)

In [6]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- V

In [7]:
df.show(1)

24/10/31 17:27:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+-------+--------+-------------------+-------------------+---------+----------+-------+-------+------------+--------------------+------+------+----------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
| ID| Source|Severity|         Start_Time|           End_Time|Start_Lat| Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|Street|  City|    County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|T

In [8]:
print(f"Number of rows: {df.count()}")
print(f"Number of columns: {len(df.columns)}")

Number of rows: 7728394
Number of columns: 46


### Count null values

In [16]:
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).first()
null_counts_list = [(df.columns[i], null_counts[i]) for i in range(len(df.columns))]

In [17]:
null_counts_list

[('ID', 0),
 ('Source', 0),
 ('Severity', 0),
 ('Start_Time', 0),
 ('End_Time', 0),
 ('Start_Lat', 0),
 ('Start_Lng', 0),
 ('End_Lat', 3402762),
 ('End_Lng', 3402762),
 ('Distance(mi)', 0),
 ('Description', 5),
 ('Street', 10869),
 ('City', 253),
 ('County', 0),
 ('State', 0),
 ('Zipcode', 1915),
 ('Country', 0),
 ('Timezone', 7808),
 ('Airport_Code', 22635),
 ('Weather_Timestamp', 120228),
 ('Temperature(F)', 163853),
 ('Wind_Chill(F)', 1999019),
 ('Humidity(%)', 174144),
 ('Pressure(in)', 140679),
 ('Visibility(mi)', 177098),
 ('Wind_Direction', 175206),
 ('Wind_Speed(mph)', 571233),
 ('Precipitation(in)', 2203586),
 ('Weather_Condition', 173459),
 ('Amenity', 0),
 ('Bump', 0),
 ('Crossing', 0),
 ('Give_Way', 0),
 ('Junction', 0),
 ('No_Exit', 0),
 ('Railway', 0),
 ('Roundabout', 0),
 ('Station', 0),
 ('Stop', 0),
 ('Traffic_Calming', 0),
 ('Traffic_Signal', 0),
 ('Turning_Loop', 0),
 ('Sunrise_Sunset', 23246),
 ('Civil_Twilight', 23246),
 ('Nautical_Twilight', 23246),
 ('Astronomica

In [21]:
len(null_counts_list)

46

### Dropping selected columns

In [24]:
drop_col = ["End_Lat", "End_Lng", "Airport_Code", "Wind_Chill(F)", "Precipitation(in)", "Wind_Speed(mph)"]
df_updated = df.drop(*drop_col)

In [27]:
unique_countries = df_updated.select("Country").distinct()
unique_countries.show()

+-------+
|Country|
+-------+
|     US|
+-------+



In [72]:
df_updated = df_updated.drop(*["Country", "Zipcode", "Timezone", "Wind_Direction", "Nautical_Twilight", "Astronomical_Twilight", "Weather_Timestamp"])

In [48]:
df_updated.select("Turning_Loop").distinct().show()

+------------+
|Turning_Loop|
+------------+
|       false|
+------------+



In [49]:
df_updated = df_updated.drop("Turning_Loop")

In [52]:
df_updated.select("Source").distinct().show()

+-------+
| Source|
+-------+
|Source3|
|Source2|
|Source1|
+-------+



In [53]:
df_updated = df_updated.drop("Source")

In [55]:
df_updated.select("Sunrise_Sunset").distinct().show()

+--------------+
|Sunrise_Sunset|
+--------------+
|         Night|
|           Day|
|          NULL|
+--------------+



In [56]:
df_updated.select("Civil_Twilight").distinct().show()

+--------------+
|Civil_Twilight|
+--------------+
|         Night|
|           Day|
|          NULL|
+--------------+



In [59]:
df_updated = df_updated.drop("Civil_Twilight")

In [73]:
df_updated.columns

['ID',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Temperature(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Weather_Condition',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Sunrise_Sunset']

In [79]:
df_updated.select("Weather_Condition").distinct().show()

+--------------------+
|   Weather_Condition|
+--------------------+
|         Ice Pellets|
|         Shallow Fog|
|        Thunderstorm|
|        Volcanic Ash|
|   N/A Precipitation|
|Showers in the Vi...|
|              Cloudy|
| Light Freezing Rain|
|        Blowing Snow|
|  Heavy Rain / Windy|
|   Low Drifting Snow|
|Heavy Thunderstor...|
|     Widespread Dust|
|         Snow Grains|
|Light Rain with T...|
|             Squalls|
|    Scattered Clouds|
|       Heavy T-Storm|
|      Patches of Fog|
|        Rain Showers|
+--------------------+
only showing top 20 rows



### Count nulls in updated dataset

In [74]:
null_counts = df_updated.select([count(when(col(c).isNull(), c)).alias(c) for c in df_updated.columns]).first()
null_counts_list = [(df_updated.columns[i], null_counts[i]) for i in range(len(df_updated.columns))]

In [75]:
null_counts_list

[('ID', 0),
 ('Severity', 0),
 ('Start_Time', 0),
 ('End_Time', 0),
 ('Start_Lat', 0),
 ('Start_Lng', 0),
 ('Distance(mi)', 0),
 ('Description', 0),
 ('Street', 0),
 ('City', 0),
 ('County', 0),
 ('State', 0),
 ('Temperature(F)', 0),
 ('Humidity(%)', 0),
 ('Pressure(in)', 0),
 ('Visibility(mi)', 0),
 ('Weather_Condition', 0),
 ('Amenity', 0),
 ('Bump', 0),
 ('Crossing', 0),
 ('Give_Way', 0),
 ('Junction', 0),
 ('No_Exit', 0),
 ('Railway', 0),
 ('Roundabout', 0),
 ('Station', 0),
 ('Stop', 0),
 ('Traffic_Calming', 0),
 ('Traffic_Signal', 0),
 ('Sunrise_Sunset', 0)]

In [119]:
df_updated = df_updated.drop("Description")

### Remove rows with null values

In [77]:
df_updated = df_updated.dropna()

In [78]:
print(f"Count of rows after removing rows with null values: {df_updated.count()}")

Count of rows after removing rows with null values: 7461669


### Convert Temperature column to Celsius

In [85]:
df_updated = df_updated.withColumn("Temperature(C)", (col("Temperature(F)") - 32) * 5 / 9)

In [86]:
df_updated.select("Temperature(F)", "Temperature(C)").show()

+--------------+------------------+
|Temperature(F)|    Temperature(C)|
+--------------+------------------+
|          36.9|2.7222222222222214|
|          37.9| 3.277777777777777|
|          36.0|2.2222222222222223|
|          35.1| 1.722222222222223|
|          36.0|2.2222222222222223|
|          37.9| 3.277777777777777|
|          34.0|1.1111111111111112|
|          34.0|1.1111111111111112|
|          33.3|0.7222222222222207|
|          37.4| 2.999999999999999|
|          35.6| 2.000000000000001|
|          37.4| 2.999999999999999|
|          33.8|0.9999999999999984|
|          36.0|2.2222222222222223|
|          37.4| 2.999999999999999|
|          33.8|0.9999999999999984|
|          35.6| 2.000000000000001|
|          36.0|2.2222222222222223|
|          37.4| 2.999999999999999|
|          36.0|2.2222222222222223|
+--------------+------------------+
only showing top 20 rows



In [91]:
df_updated = df_updated.drop("Temperature(F)")

In [89]:
df_updated.select(
    min("Temperature(C)").alias("Min (C)"),
    max("Temperature(C)").alias("Max (C)")
).show()

+------------------+-------+
|           Min (C)|Max (C)|
+------------------+-------+
|-42.77777777777778|   95.0|
+------------------+-------+



#### Based on temperature min max results:
- #### There aren't outliers with minus temperatures because the coldest temperature ever recorded in the US was -80°C in Alaska
- #### There are outliers with plus temperatures because the hottest temperature ever recorded in the US was 56.7°C in California

In [94]:
df_updated = df_updated.filter(col("Temperature(C)") <= 56.7)

In [98]:
df_updated = df_updated.withColumn("Temperature(C)", round(col("Temperature(C)"), 1))

In [99]:
df_updated.select(
    min("Temperature(C)").alias("Min (C)"),
    max("Temperature(C)").alias("Max (C)")
).show()

+-------+-------+
|Min (C)|Max (C)|
+-------+-------+
|  -42.8|   56.0|
+-------+-------+



### Check humidity values

In [100]:
df_updated.select(
    min("Humidity(%)").alias("Min Humidity(%)"),
    max("Humidity(%)").alias("Max Humidity(%)")
).show()

+---------------+---------------+
|Min Humidity(%)|Max Humidity(%)|
+---------------+---------------+
|            1.0|          100.0|
+---------------+---------------+



### Check Pressure(in) values

In [101]:
df_updated.select(
    min("Pressure(in)").alias("Min Pressure(in)"), 
    max("Pressure(in)").alias("Max Pressure(in)")
).show()

+----------------+----------------+
|Min Pressure(in)|Max Pressure(in)|
+----------------+----------------+
|             0.0|           58.63|
+----------------+----------------+



#### Pressure(in) column has outliers because:
- #### Lowest ever recorded air pressure 25.69 in
- #### Highest ever recorded air pressure 32.01 in

In [103]:
df_updated.filter((col("Pressure(in)") < 25.69) | (col("Pressure(in)") > 32.01)).count()

162051

#### Considering that there are a lot of outliers based on this parameter, let's cap them insead of dropping

In [104]:
df_updated = df_updated.withColumn("Pressure(in)",
    when(col("Pressure(in)") < 25.69, 25.69)
    .when(col("Pressure(in)") > 32.01, 32.01)
    .otherwise(col("Pressure(in)"))
)

In [105]:
df_updated.select(
    min("Pressure(in)").alias("Min Pressure(in)"), 
    max("Pressure(in)").alias("Max Pressure(in)")
).show()

+----------------+----------------+
|Min Pressure(in)|Max Pressure(in)|
+----------------+----------------+
|           25.69|           32.01|
+----------------+----------------+



### Check Visibility(mi) values

In [106]:
df_updated.select(
    min("Visibility(mi)").alias("Visibility(mi)"), 
    max("Visibility(mi)").alias("Visibility(mi)")
).show()

+--------------+--------------+
|Visibility(mi)|Visibility(mi)|
+--------------+--------------+
|           0.0|         140.0|
+--------------+--------------+



#### Converting Visibility in miles to Visibility in kilometers

In [107]:
df_updated = df_updated.withColumn("Visibility(km)", col("Visibility(mi)") * 1.60934)

In [108]:
df_updated.select("Visibility(km)", "Visibility(mi)").show()

+--------------+--------------+
|Visibility(km)|Visibility(mi)|
+--------------+--------------+
|       16.0934|          10.0|
|       16.0934|          10.0|
|       16.0934|          10.0|
|      14.48406|           9.0|
|       9.65604|           6.0|
|      11.26538|           7.0|
|      11.26538|           7.0|
|      11.26538|           7.0|
|        8.0467|           5.0|
|       4.82802|           3.0|
|        8.0467|           5.0|
|       4.82802|           3.0|
|       4.82802|           3.0|
|       16.0934|          10.0|
|       4.82802|           3.0|
|       4.82802|           3.0|
|      11.26538|           7.0|
|       16.0934|          10.0|
|       16.0934|          10.0|
|       16.0934|          10.0|
+--------------+--------------+
only showing top 20 rows



In [109]:
df_updated = df_updated.drop("Visibility(mi)")

### Check Distance(mi) values

In [110]:
df_updated.select(
    min("Distance(mi)").alias("Distance(mi)"), 
    max("Distance(mi)").alias("Distance(mi)")
).show()

+------------+------------+
|Distance(mi)|Distance(mi)|
+------------+------------+
|         0.0|      441.75|
+------------+------------+



#### Converting Distance in miles to Distance in kilometers

In [111]:
df_updated = df_updated.withColumn("Distance(km)", col("Distance(mi)") * 1.60934)

In [112]:
df_updated.select("Distance(km)", "Distance(mi)").show()

+------------+------------+
|Distance(km)|Distance(mi)|
+------------+------------+
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|         0.0|         0.0|
|   0.0160934|        0.01|
|         0.0|         0.0|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|         0.0|         0.0|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
|         0.0|         0.0|
|   0.0160934|        0.01|
|   0.0160934|        0.01|
+------------+------------+
only showing top 20 rows



In [113]:
df_updated = df_updated.drop("Distance(mi)")

### Check Time columns values

#### Check if Time columns have proper format

In [115]:
df_updated.filter(~col("Start_Time").cast("timestamp").isNotNull()).count()

0

In [117]:
df_updated.filter(~col("End_Time").cast("timestamp").isNotNull()).count()

0

#### Check if there are cases where Start_Time is after End_Time

In [116]:
df_updated.filter(col("Start_Time") > col("End_Time")).count()

0

In [118]:
df_updated.select("Start_Time", "End_Time").show()

+-------------------+-------------------+
|         Start_Time|           End_Time|
+-------------------+-------------------+
|2016-02-08 05:46:00|2016-02-08 11:00:00|
|2016-02-08 06:07:59|2016-02-08 06:37:59|
|2016-02-08 06:49:27|2016-02-08 07:19:27|
|2016-02-08 07:23:34|2016-02-08 07:53:34|
|2016-02-08 07:39:07|2016-02-08 08:09:07|
|2016-02-08 07:44:26|2016-02-08 08:14:26|
|2016-02-08 07:59:35|2016-02-08 08:29:35|
|2016-02-08 07:59:58|2016-02-08 08:29:58|
|2016-02-08 08:00:40|2016-02-08 08:30:40|
|2016-02-08 08:10:04|2016-02-08 08:40:04|
|2016-02-08 08:14:42|2016-02-08 08:44:42|
|2016-02-08 08:21:27|2016-02-08 08:51:27|
|2016-02-08 08:36:34|2016-02-08 09:06:34|
|2016-02-08 08:37:07|2016-02-08 09:07:07|
|2016-02-08 08:39:43|2016-02-08 09:09:43|
|2016-02-08 08:43:20|2016-02-08 09:13:20|
|2016-02-08 08:53:17|2016-02-08 09:23:17|
|2016-02-08 09:24:37|2016-02-08 09:54:37|
|2016-02-08 09:25:17|2016-02-08 09:55:17|
|2016-02-08 09:35:35|2016-02-08 10:05:35|
+-------------------+-------------

In [120]:
df_updated.columns

['ID',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'Street',
 'City',
 'County',
 'State',
 'Humidity(%)',
 'Pressure(in)',
 'Weather_Condition',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Sunrise_Sunset',
 'Temperature(C)',
 'Visibility(km)',
 'Distance(km)']

### Checking number of duplicates in dataset

In [123]:
duplicate_counts = df_updated.groupBy(df_updated.columns).count()
duplicate_counts.filter(col("count") > 1).count()

24/10/31 21:55:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/31 21:55:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/31 21:55:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/31 21:55:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/31 21:55:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/31 21:55:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/31 21:55:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/31 21:55:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/31 21:56:06 WARN RowBasedKeyValueBatch: Calling spill() on

0